# --Eğitim bölümü. Direkt kullanım bölümüne geçmek için aşağı inin.

In [1]:
%%capture
!pip install -q sentence-transformers datasets pandas scikit-learn faiss-cpu gradio stopwords_tr

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, AutoModelForSeq2SeqLM
import torch
from datasets import load_dataset
from transformers import Trainer, TrainingArguments
from huggingface_hub import login
import pandas as pd
from sentence_transformers import SentenceTransformer
import gradio as gr
import html
from sklearn.metrics.pairwise import cosine_similarity
import faiss
import pickle
import numpy as np
from os import makedirs, path, environ

environ["HF_DATASETS_CACHE"] = "./hf_cache/datasets"
environ["HF_HOME"] = "./hf_cache"

print("ok")

ok


In [6]:
# eğitim (huggingface token ve verisetine erişim gerekiyor).
# verisetini indirmek için hf'de onay vermeniz gerekiyor. https://huggingface.co/datasets/alibayram/hukuk_soru_cevap/tree/main
login()

makedirs("artifacts", exist_ok=True)

dataset = load_dataset(
    "alibayram/hukuk_soru_cevap",
    split="train",
)

embed_model = SentenceTransformer(
    "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
)

#embed
sorular = [str(s) for s in dataset["soru"]]
cevaplar = [str(c) for c in dataset["cevap"]]

embeddings = embed_model.encode(
    sorular,
    convert_to_numpy=True,
    normalize_embeddings=True
)

dim = embeddings.shape[1]
index = faiss.IndexFlatIP(dim)
index.add(embeddings)

#kayıt
makedirs("artifacts", exist_ok=True)
faiss.write_index(index, "artifacts/faiss.index")
with open("artifacts/sorular.pkl", "wb") as f:
    pickle.dump(sorular, f)

with open("artifacts/cevaplar.pkl", "wb") as f:
    pickle.dump(cevaplar, f)

print("ok")

In [7]:
#veriseti yüklenme kontrolü
df = pd.DataFrame(dataset)

df = df[["soru", "cevap"]].dropna()
df = df.reset_index(drop=True)

print("yüklenen örnek sayısı:", len(df))
df.head()

print("ok")

yüklenen örnek sayısı: 2080
ok


In [8]:
questions = df["soru"].tolist()
dim = embeddings.shape[1]

index = faiss.IndexFlatIP(dim)
faiss.normalize_L2(embeddings)

index.add(embeddings)


# --test bloğu (bunun üstündekileri çalıştırmanıza gerek yok. Eğitim yapmadıysanız önce artifact dosyasının içeriğini sol taraftaki klasör simgesine yükleyin.)

In [9]:
%%capture
!pip install sentence-transformers faiss-cpu gradio stopwords_tr

In [10]:
import stopwords_tr as stp
from sentence_transformers import SentenceTransformer
import faiss
import pickle
from os import path
import gradio as gr
import html

embed_model = SentenceTransformer(
    "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
)


if path.exists("/content/artifacts/faiss.index"):
    embed_yolu = "/content/artifacts"
elif path.exists("/content/faiss.index"):
    embed_yolu = "/content"
else:
    raise FileNotFoundError("Dosyalar bulunamadı. faiss.index, sorular.pkl ve cevaplar.pkl dosyalarını yüklediğinizden emin olun.")

index = faiss.read_index(path.join(embed_yolu, "faiss.index"))

with open(path.join(embed_yolu, "sorular.pkl"), "rb") as f:
    sorular = pickle.load(f)

with open(path.join(embed_yolu, "cevaplar.pkl"), "rb") as f:
    cevaplar = pickle.load(f)

print(f"{len(sorular)} adet soru yüklendi.")

#cevap alma fonksiyonu
def metin_temizle(text):
    #basit önişleme
    text = text.replace("İ", "i").replace("I", "ı").lower()
    for punc in [".", ",", "?", "!", ":", ";", "(", ")", "\"", "'", "-"]:
        text = text.replace(punc, " ")

    kelimeler = text.split()
    temiz_kelimeler = [k for k in kelimeler if not k in stp.stopwords() and len(k) > 2]

    return temiz_kelimeler

def sonuc_getir(soru, top_k=5, min_score=0.45):
    #stopwordsları çıkarıyoruz benzerlik hesaplama için
    temiz_kelimeler = metin_temizle(soru)
    n_temiz_kelime = len(temiz_kelimeler)

    # kelime eşleşme skoru için aday havuzu geniş
    query_vec = embed_model.encode([soru], convert_to_numpy=True, normalize_embeddings=True)
    adaylar = 50
    scores, i = index.search(query_vec, adaylar)

    sonuc = []

    for score, idx in zip(scores[0], i[0]):
        idx = int(idx)
        db_soru = sorular[idx]
        db_cevap = cevaplar[idx]

        Bscore = float(score)

        # Temizlik
        db_soru = db_soru.replace("İ", "i").replace("I", "ı").lower()

        # Kelime Eşleşmesi Sayma
        matchs = 0
        for kelime in temiz_kelimeler:
            if kelime in db_soru:
                matchs += 1

        if n_temiz_kelime > 0:
            #sorudaki kelimelerin ne kadarı geçiyorsa puan o kadar artırılıyor
            if matchs > 0:
                Bscore = Bscore + (matchs / n_temiz_kelime * 0.30)
            #ortak kelime yoksa puan düşür
            elif Bscore < 0.40:
                Bscore = Bscore * 0.8

        # eleme
        if Bscore < min_score:
            continue

        sonuc.append({
            "soru": db_soru,
            "cevap": db_cevap,
            "score": min(Bscore, 1.0),
            "matches": matchs
        })

    # sıralama
    sonuc = sorted(sonuc, key=lambda x: x["score"], reverse=True)
    return sonuc[:top_k]

#arayüz
sorusayisi = 5 #sorulan soruya en fazla kaç yanıt gösterilsin

def forum_asistani(soru):
    if not soru or not soru.strip():
        return "Bir soru girin."

    try:
        sonuc = sonuc_getir(soru, top_k=sorusayisi, min_score=0.45)
    except NameError:
        return "Bir hata oluştu. önceki hücreleri tekrar başlatmayı deneyin."

    if sonuc is None:
        return "**Bu soru için yeterli bilgi bulunmamaktadır.**\n\n"

    # sonuç gösterme
    md = []
    md.append(f"# Sonuçlar \n")
    md.append(f"**Orijinal Soru:** {html.escape(soru)}\n\n---\n")

    for i, r in enumerate(sonuc, start=1):
        matched_q = html.escape(r.get("soru", ""))
        answer = r.get("cevap", "")
        score = min(r.get("score", 0.0), 1.0)

        section = (
            f"### Sonuç {i}  (Benzerlik: {score:.2f})\n\n"
            f"**Cevap verilen soru:** {matched_q}\n\n"
            f"**Cevap:** {answer}\n\n"
            f"**Kaynak:** [İlgili soruya gitmek için buraya tıklayın](#)\n\n"
            "---\n"
        )
        md.append(section)
    return "\n".join(md)


demo = gr.Interface(
    fn=forum_asistani,
    allow_flagging="never",
    inputs=[
        gr.Textbox(lines=4, placeholder="Sorunuzu buraya yazın \n(ör: \"taşınmazda vergi borcu\" veya \"nikah\")", label="Soru"),
    ],

    outputs=gr.Markdown(label="Cevaplar"),
    title="Cevap Asistanı - Tamer BAYAR 220205027",
    description="Daha önce sorulmuş sorulardan benzer olanları bulup getirir."
)

demo.launch(share=True)


2080 adet soru yüklendi.


/usr/local/lib/python3.12/dist-packages/gradio/interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://578e3decda40c36521.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# --diğer

In [ ]:
#arayüzsüz demo

while True:
    q = input("Soru (çıkmak için q): ")
    if q.lower() == "q":
        break

    results = sonuc_getir(q)

    if results is None:
        print("\nBu soru için yeterli bilgi bulunmamaktadır.\n")
        continue

    for i, r in enumerate(results, 1):
        print(f"\n🔹 Sonuç {i} | Benzerlik: {r['score']:.3f}")
        print("Cevap:")
        print(r["cevap"])


In [12]:
# standalone test - bir girdi ile bir sorgunun benzerliğini kolayca ölçer
!pip install stopwords_tr

import stopwords_tr as stp
from sentence_transformers import SentenceTransformer
import numpy as np

embed_model = SentenceTransformer(
    "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
)

def metin_temizle(text):
    text = text.replace("İ", "i").replace("I", "ı").lower()
    for punc in [".", ",", "?", "!", ":", ";", "(", ")", "\"", "'", "-"]:
        text = text.replace(punc, " ")

    kelimeler = text.split()
    temiz_kelimeler = [k for k in kelimeler if k not in stp.stopwords() and len(k) > 2]
    return temiz_kelimeler

def benzerlik_hesapla(sorgu, konu):
    vecs = embed_model.encode(
        [sorgu, konu],
        convert_to_numpy=True,
        normalize_embeddings=True
    )

    base_score = float(np.dot(vecs[0], vecs[1]))

    temiz_kelimeler = metin_temizle(sorgu)
    n_temiz = len(temiz_kelimeler)

    konu_clean = konu.replace("İ", "i").replace("I", "ı").lower()

    matches = sum(1 for k in temiz_kelimeler if k in konu_clean)

    Bscore = base_score

    if n_temiz > 0:
        if matches > 0:
            Bscore = Bscore + (matches / n_temiz * 0.30)
        elif Bscore < 0.40:
            Bscore = Bscore * 0.8

    return {
        "sorgu": sorgu,
        "konu": konu,
        "base_score": round(base_score, 4),
        "final_score": round(min(Bscore, 1.0), 4),
        "matches": matches,
        "temiz_kelimeler": temiz_kelimeler
    }

konu  = "The transfer of property ownership requires a formal deed at the land registry office."
sorgu = "Tapu dairesinde mülkiyet devri nasıl yapılır?"

sonuc = benzerlik_hesapla(sorgu, konu)

print("\n")
print(f"sorgu : {sonuc['sorgu']}")
print(f"soru : {sonuc['konu']}")
print(f"temiz kelimeler : {sonuc['temiz_kelimeler']}")
print(f"eşleşen kelimeler: {sonuc['matches']}")
print(f"vektör skoru : {sonuc['base_score']}")
print(f"final skor : {sonuc['final_score']}")




sorgu : Tapu dairesinde mülkiyet devri nasıl yapılır?
soru : The transfer of property ownership requires a formal deed at the land registry office.
temiz kelimeler : ['tapu', 'dairesinde', 'mülkiyet', 'devri', 'yapılır']
eşleşen kelimeler: 0
vektör skoru : 0.4614
final skor : 0.4614


In [ ]:
#directory bulunamadı hatası alırsanız önce bu hücreyi çalıştırın, olmazsa çalışma zamanı bağlantısını kesip tekrar bağlamayı deneyin.
!rm -rf /root/.cache/huggingface/datasets
!rm -rf /root/.cache/huggingface/hub